In [141]:
import numpy as np
import re
import tensorflow as tf


In [5]:
lines=open('movie_lines.txt',encoding='utf-8',errors='ignore').read()
conversation=open('movie_conversations.txt',encoding='utf-8',errors='ignore').read()

In [7]:
lines=lines.split('\n')
conversation=conversation.split('\n')

In [44]:
lines[-1:0]

[]

In [9]:
conversation[:5]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']"]

### We will create a dict with only line id and line text

In [45]:
line_id={}
for each_line in lines:
    line_descr = each_line.split(' +++$+++ ')
    if len(line_descr) == 5:
        line_id[line_descr[0]] = line_descr[4]

In [46]:
line_id['L1045']

'They do not!'

In [48]:
convo_with_line_id=[]
for convo in conversation:
    convo_list=convo.split(' +++$+++ ')
    if len(convo_list) == 4:
        convo_with_line_id.append(convo_list[3])

In [49]:
convo_with_line_id[:5]

["['L194', 'L195', 'L196', 'L197']",
 "['L198', 'L199']",
 "['L200', 'L201', 'L202', 'L203']",
 "['L204', 'L205', 'L206']",
 "['L207', 'L208']"]

In [50]:
## WE will clean all those string related in the above list

In [88]:
convo_with_line_id_refined=[]
for convo_list in convo_with_line_id:
    #removingthe quote and list symbols
    trimming=convo_list[2:-2]
    #print(trimming)
    trimming=trimming.replace("'", "")
    individual_lines=trimming.replace(" ","")
    #print(individual_lines)
    individual_lines=individual_lines.split(',')
    #print(individual_lines)
    convo_with_line_id_refined.append(individual_lines)
    

In [89]:
convo_with_line_id_refined[:5]

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208']]

### Now we have got the list of conversations and lineids related to convos. Now we will create questions and answers from these

In [90]:
questions=[]
answers=[]

In [91]:
for convo_list in convo_with_line_id_refined:
    for line_code in range(len(convo_list)-1):
        questions.append(line_id[convo_list[line_code]])
        answers.append(line_id[convo_list[line_code +1]])

In [93]:
questions[:10]

['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
 "Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "You're asking me out.  That's so cute. What's your name again?",
 "No, no, it's my fault -- we didn't have a proper introduction ---",
 'Cameron.',
 "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 'Why?',
 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 'Gosh, if only we could find Kat a boyfriend...']

#### We got questions and answers. Now we have to clean the data formm Q&A

In [179]:
def clean_data(text):
    #lower case
    text=text.lower()
    text = re.sub(r"i'm", "i am",text)
    text= re.sub(r"he's","he is",text)
    text=re.sub(r"she's", "she is",text)
    text=re.sub(r"that's","that is",text)
    text=re.sub(r"what's","what is",text)
    text=re.sub(r"where's","where is",text)
    text=re.sub(r"\'ll"," will",text)
    text=re.sub(r"\'ve"," have",text)
    text=re.sub(r"\'d'"," would",text)
    text=re.sub(r"\'re"," are",text)
    text=re.sub(r"won't","will not",text)
    text=re.sub(r"can't","cannot",text)
    text=re.sub(r"how'd","how would",text)
    text=re.sub(r"[-()\"#/@;:<>{}+=~|.?,]","",text)
    #3 We will remove some extra spaces
    text=text.replace(5*' ',' ')
    text=text.replace(4*' ',' ')
    text=text.replace(3*' ',' ')
    text=text.replace(2*' ',' ')
    #print(text)
    return text

In [180]:
clean_ques=[]
clean_ans=[]
for text in questions:
    #print(text)
    clean_text=clean_data(text)
    clean_ques.append(clean_text)

In [181]:
for text in answers:
    
    clean_text=clean_data(text)
    #print(text)
    clean_ans.append(clean_text)

In [182]:
clean_ans[125:135]

['fine i see that i am a prisoner in my own house i am not a daughter i am a possession!',
 "i have a date daddy and he ' s not a captain of oppression like some men we know",
 "didn't have you pegged for a gigglepuss fan aren't they a little too preteen bellybutton ring for you",
 'fan of a fan you see a couple of minors come in',
 'never',
 'padua girls one tall decent body the other one kinda short and undersexed',
 "just sent 'em through",
 'so they tell me',
 "c'mon i am supposed to give you the tour",
 'north actually how would you ']

### Now we will take words and give them a unique id of allthe words
#### For that we get all the words and their count from questions and answers and then removeless frequent words

In [184]:
wordscount={}

In [189]:
for line in clean_ques:
    for word in line.split():
        if word not in wordscount:
            wordscount[word] = 1
        else:
            wordscount[word] += 1
for line in clean_ans:
    for word in line.split():
        if word not in wordscount:
            wordscount[word] = 1
        else:
            wordscount[word] += 1

### Now we will take out the less frequent words from the count

In [195]:
threshold=20
word_filetered_words=[]

In [197]:
for word,count, in wordscount.items():
    if count > threshold:
        word_filetered_words.append(word)

In [198]:
word_filetered_words[:10]

['can', 'we', 'make', 'this', 'quick', 'and', 'andrew', 'are', 'having', 'an']

### Now we have all the unique words and we will assign unique id's to all the words

In [199]:
unique_word_id={}

In [200]:
word_count=0
for word in word_filetered_words:
    unique_word_id[word] = word_count
    word_count += 1

In [203]:
unique_word_id['fine']

533

#### We will create tokens and we will assign to unique id


In [205]:
tokens=['<PAD>','<EOS>','<OUT>','<SOS>']
for token in tokens:
    unique_word_id[token] =len(unique_word_id) + 1

In [206]:
unique_word_id['<SOS>']

8554

#### We will create two identiacal dicts from the above dict as we need to make some different changes according to question and answer

In [207]:
unique_ans=unique_word_id
unique_ques=unique_word_id

### We will append EOS string to every end of answer 

In [208]:
eos_append_ans=[]
for sentence in clean_ans:
    eos_append_ans.append(sentence+' <EOS>')

In [209]:
eos_append_ans[:10]

["well i thought we'd start with pronunciation if that is okay with you <EOS>",
 'not the hacking and gagging and spitting part please <EOS>',
 "okay then how 'bout we try out some french cuisine saturday night <EOS>",
 'forget it <EOS>',
 'cameron <EOS>',
 'the thing is cameron i am at the mercy of a particularly hideous breed of loser my sister i cannot date until she does <EOS>',
 'seems like she could get a date easy enough <EOS>',
 'unsolved mystery she used to be really popular when she started high school then it was just like she got sick of it or something <EOS>',
 'that is a shame <EOS>',
 'let me see what i can do <EOS>']

##### We now will convert the question and answer to integers as we have the reference of words to integers and if we find any word which is not present(Because we have removed some non-frequent words) we will replace that with the id of OUT

In [217]:
num_ques=[]
num_ans=[]

In [218]:
for sentence in clean_ques:
    ints=[]
    for word in sentence.split():
        if word not in unique_ques:
            ints.append(unique_ques['<OUT>'])
        else:
            ints.append(unique_ques[word])
    num_ques.append(ints)

In [219]:
## Here we will loop the eos_append_ans as we have added EOS
for sentence in eos_append_ans:
    ints=[]
    for word in sentence.split():
        if word not in unique_ans:
            ints.append(unique_ans['<OUT>'])
        else:
            ints.append(unique_ans[word])
    num_ans.append(ints)

In [222]:
num_ans[3]

[245, 74, 8552]

### Now we have converted the words into unique integers.
### Before training we will sort the lists from less words to more words
#### For quicker training we will avoid sentences which has words more than 30

In [225]:
sorted_ques=[]
sorted_ans=[]

In [226]:
for i in range(1,31):
        for index,sentence in enumerate(num_ques):
             if len(sentence) == i:
                    sorted_ques.append(num_ques[index])
                    sorted_ans.append(num_ans[index])

In [230]:
sorted_ques[0]

[47]

In [229]:
sorted_ans[0]

[15,
 48,
 25,
 47,
 18,
 49,
 50,
 15,
 51,
 52,
 45,
 53,
 8553,
 54,
 52,
 55,
 41,
 56,
 18,
 57,
 58,
 59,
 60,
 61,
 8552]